In [11]:
import requests
import os

workspace = r'C:\School\GIS 5572\Local\Lab 2\data'

weather_data = 'https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&year=2024&month=1'

response = requests.get(weather_data)
file_path = os.path.join(workspace, 'weather_jan.geojson')
with open(file_path, 'wb') as file:
    file.write(response.content)

In [45]:
out_features = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\Test Project.gdb\weather_jan'
arcpy.conversion.JSONToFeatures(file_path, out_features, geometry_type="POINT")

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\Test Project.gdb\\weather_jan'>

In [46]:
desc = arcpy.Describe(out_features)

# Expected bounding box values
expected_xmin = -98
expected_ymin = 43
expected_xmax = -89
expected_ymax = 49

# Expected spatial reference name
expected_spatial_reference = 'GCS_WGS_1984'

# Print the bounding box
print("Bounding Box:")
print("XMin: {0}, YMin: {1}, XMax: {2}, YMax: {3}".format(desc.extent.XMin, desc.extent.YMin, desc.extent.XMax, desc.extent.YMax))

# Print the spatial reference
print("\nSpatial Reference:")
print(desc.spatialReference.name)

# Assert statements to check the values
assert desc.extent.XMin >= expected_xmin, f"XMin does not match. Expected: {expected_xmin}, Actual: {desc.extent.XMin}"
assert desc.extent.YMin >= expected_ymin, f"YMin does not match. Expected: {expected_ymin}, Actual: {desc.extent.YMin}"
assert desc.extent.XMax <= expected_xmax, f"XMax does not match. Expected: {expected_xmax}, Actual: {desc.extent.XMax}"
assert desc.extent.YMax <= expected_ymax, f"YMax does not match. Expected: {expected_ymax}, Actual: {desc.extent.YMax}"
assert desc.spatialReference.name == expected_spatial_reference, f"Spatial reference does not match. Expected: {expected_spatial_reference}, Actual: {desc.spatialReference.name}"

print("\nAll values match the expected values.")

Bounding Box:
XMin: -97.20169830299994, YMin: 43.508331299000076, XMax: -89.68496704099994, YMax: 48.970851898000035

Spatial Reference:
GCS_WGS_1984

All values match the expected values.


In [47]:
import datetime

bad_data_ids = []

# Check the data points
with arcpy.da.SearchCursor(out_features, ["OBJECTID", "date", "min_tmpf", "max_tmpf"]) as cursor:
    for row in cursor:
        obj_id = row[0]
        date_col = row[1]
        min_temp = row[2]
        max_temp = row[3]
        
        date = datetime.datetime.strptime(date_col, '%Y-%m-%d')
        is_january = date.month == 1
        if min_temp is None:
            is_min_valid = False
        else: 
            is_min_valid = -30 <= min_temp <= 61
        if max_temp is None:
            is_max_valid = False
        else: 
            is_max_valid = -30 <= max_temp <= 61

        # Check if the date is within January, max_tmpf is betwen -20 and 61, and min_tmpf is between -20 and 61 
        if not is_january:
            print(f"Incorrect date, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        elif not is_min_valid:
            print(f"Incorrect min_tmpf, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        elif not is_max_valid:
            print(f"Incorrect max_tmpf, ObjectID: {obj_id}")
            bad_data_ids.append(obj_id)
        
print(bad_data_ids)   

Incorrect min_tmpf, ObjectID: 58
Incorrect min_tmpf, ObjectID: 221
Incorrect min_tmpf, ObjectID: 384
Incorrect min_tmpf, ObjectID: 547
Incorrect min_tmpf, ObjectID: 2376
Incorrect min_tmpf, ObjectID: 2539
Incorrect min_tmpf, ObjectID: 2616
Incorrect min_tmpf, ObjectID: 2702
Incorrect min_tmpf, ObjectID: 2779
Incorrect min_tmpf, ObjectID: 2865
Incorrect min_tmpf, ObjectID: 3028
Incorrect min_tmpf, ObjectID: 3191
Incorrect min_tmpf, ObjectID: 3354
Incorrect min_tmpf, ObjectID: 3452
Incorrect min_tmpf, ObjectID: 3517
Incorrect min_tmpf, ObjectID: 3680
Incorrect min_tmpf, ObjectID: 4117
[58, 221, 384, 547, 2376, 2539, 2616, 2702, 2779, 2865, 3028, 3191, 3354, 3452, 3517, 3680, 4117]


In [55]:
# Remove unwanted data
arcpy.env.workspace = r'C:\School\GIS 5572\Lab 1.2\Test Project\Test_Project'
out_features = 'weather_jan'
where_clause = "OBJECTID IN ({})".format(", ".join(map(str, bad_data_ids)))
print(where_clause)
arcpy.SelectLayerByAttribute_management(out_features, "NEW_SELECTION", where_clause)
arcpy.DeleteFeatures_management(out_features)

OBJECTID IN (58, 221, 384, 547, 2376, 2539, 2616, 2702, 2779, 2865, 3028, 3191, 3354, 3452, 3517, 3680, 4117)


<Result 'weather_jan'>

In [53]:
arcpy.env.workspace = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde'
spatial_ref = arcpy.SpatialReference(4326)
out_name = "weather_jan"
geometry_type = "POINT"
arcpy.CreateFeatureclass_management(arcpy.env.workspace, out_name, geometry_type, spatial_reference = spatial_ref)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\weather_jan'>

In [57]:
in_features = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\Test Project.gdb\weather_jan'
out_features = r'C:\School\GIS 5572\Local\Lab 1.2\Test Project\PostgreSQL-35-gis5572(postgres).sde\weather_jan'
arcpy.management.CopyFeatures(in_features, out_features)

<Result 'C:\\School\\GIS 5572\\Local\\Lab 1.2\\Test Project\\PostgreSQL-35-gis5572(postgres).sde\\gis5572.postgres.weather_jan'>